In [8]:
%pip install neo4j graphistry[bolt]

Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
import warnings
import neo4j
from neo4j import GraphDatabase, Driver
import graphistry
import pandas as pd


warnings.filterwarnings("ignore")

NEO4J_URI = "bolt://neo4j.neo4j.svc.cluster.local:7687"
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = 'neo4j'
NEO4J_CREDS = {'uri': NEO4J_URI, 'auth': ('neo4j', NEO4J_PASSWORD)}

print('neo4j', neo4j.__version__)
print('graphistry', graphistry.__version__)

neo4j 5.19.0
graphistry 0.33.7


In [10]:
# Function to convert Neo4j results to a Pandas DataFrame
def neo4j_results_to_df(results):
    return pd.DataFrame([dict(record) for record in results])

In [11]:
driver = GraphDatabase.driver(**NEO4J_CREDS)

# Execute Cypher query and fetch results
cypher_query_old = """
MATCH (n)-[r]->(m) 
RETURN n, r, m
"""
cypher_query = """
MATCH (n)-[r]->(m)
RETURN 
    id(n) AS n_id, n.name AS n_name, n.platform AS n_platform, n.content AS n_content, n.timestamp AS n_timestamp,
    id(m) AS m_id, m.name AS m_name, m.platform AS m_platform, m.content AS m_content, m.timestamp AS m_timestamp,
    id(r) AS r_id, type(r) AS r_type, r.weight AS r_weight
"""

with driver.session() as session:
    results = session.run(cypher_query)
    # Initialize lists to store node and edge data
    nodes_data = []
    edges_data = []
    
    for record in results:
        # Process node n
        nodes_data.append({
            'id': record['n_id'], 
            'name': record['n_name'], 
            'platform': record.get('n_platform', ''), 
            'content': record.get('n_content', ''), 
            'timestamp': record.get('n_timestamp', '')
        })
        # Process node m
        nodes_data.append({
            'id': record['m_id'], 
            'name': record['m_name'], 
            'platform': record.get('m_platform', ''), 
            'content': record.get('m_content', ''), 
            'timestamp': record.get('m_timestamp', '')
        })
        # Process edge r
        edges_data.append({
            'source': record['n_id'], 
            'target': record['m_id'], 
            'type': record['r_type'], 
            'weight': record.get('r_weight', 0)
        })

# Remove duplicate nodes
nodes_df = pd.DataFrame(nodes_data).drop_duplicates(subset=['id']).reset_index(drop=True)
edges_df = pd.DataFrame(edges_data)


In [12]:
plotter = graphistry.bind(source="source", destination="target", node='id')
plotter = plotter.bind(point_title='name', edge_title='type', edge_weight='weight')
plotter = plotter.nodes(nodes_df).edges(edges_df)
plotter.plot()

HTTPError: 500 Server Error: Internal Server Error for url: https://hub.graphistry.com/etl?usertag=fc87eb7f-pygraphistry-0.33.7&agent=pygraphistry&apiversion=1&agentversion=0.33.7&key=0LTB3K16H2BAAGI9